In [16]:
import pandas
from tqdm import tqdm
import stanza
from transformers import AutoTokenizer, AutoModelForCausalLM

In [17]:
# logics for the various phrases

def remove_adjectival_phrases(text: list):
    def remove_from_sent(sentence: list):
        indices_to_be_removed = []
        phrase = []
        
        for i, entry in enumerate(sentence):
            word = entry["word"]
            label = entry["label"]
            
            if label in ["RB", "DEM", "JJ", "INTF"]:
                phrase.append(i)
            else:
                if phrase:
                    indices_to_be_removed.extend(phrase)
                    phrase = [] 
                    
        new_sent = []
        for i in range(len(sentence)):
            if i == indices_to_be_removed[0]:
                del indices_to_be_removed[0]
            else:
                new_sent.append(sentence)
                
        return new_sent
                
    return [remove_from_sent(sentence) for sentence in text]


def remove_adveribial_phrases(text):
    def remove_from_sent(sentence: list):
        indices_to_be_removed = []
        phrase = []
        
        for i, entry in enumerate(sentence):
            word = entry["word"]
            label = entry["label"]
            
            if label in ['RB', 'INTF', 'RP', 'QF', 'QCC', 'RBC']:
                phrase.append(i)
            
            elif label == 'JJ' and phrase and sentence[phrase[-1]]['label'] in ['INTF', 'RB']:
                phrase.append(i)
            
            elif phrase:
                indices_to_be_removed.extend(phrase)
                phrase = [] 
                    
        new_sent = []
        for i in range(len(sentence)):
            if i == indices_to_be_removed[0]:
                del indices_to_be_removed[0]
            else:
                new_sent.append(sentence)
                
        return new_sent
                
    return [remove_from_sent(sentence) for sentence in text]


def remove_fragments(text):
    def is_fragment(sentence):
        has_noun = False
        has_verb = False
        
        for word in sentence:
            if word['label'] in ['PRP', 'NNP', 'NN', 'NNPC', 'NNC', 'DEM', 'PRPC']:
                has_noun = True
                
            if word['label'] in ['VM', 'VAUX', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:
                has_verb = True
                
        return has_noun and has_verb
    
    return [sentence for sentence in text if not is_fragment(sentence)]   


def remove_parentheticals(text: list):
    def remove_from_sent(sentence: list):
        is_parenthetical = False
        
        for i in range(len(text)):
            if text[i]['word'] == '(':
                is_parenthetical = True
                
            if text[i]['word'] == ')':
                is_parenthetical = False
                text[i] = "deleted"
                
            if is_parenthetical:
                text[i] = "deleted"
                
        return [word for word in text if word != "deleted"]
        
    return [remove_from_sent(sentence) for sentence in text]


def remove_prepositional_phrases(text):
    def remove_from_sent(sentence: list):
        indices_to_be_removed = []
        phrase = []
        
        for i, entry in enumerate(sentence):
            word = entry["word"]
            label = entry["label"]
            
            if label in ['NN', 'NNP', 'NNC', 'NNPC', 'NNS']:
                phrase.append(i)
            elif label == 'PSP':
                if phrase:
                    phrase.append(i)
                    indices_to_be_removed.extend(phrase)
                    phrase = []
            else:
                if phrase:
                    phrase = []
                    
        new_sent = []
        for i in range(len(sentence)):
            if i == indices_to_be_removed[0]:
                del indices_to_be_removed[0]
            else:
                new_sent.append(sentence)
                
        return new_sent
                
    return [remove_from_sent(sentence) for sentence in text]


def delete_required_parts(text: list):
    new_text = remove_adveribial_phrases(text)
    new_text = remove_adjectival_phrases(new_text)
    new_text = remove_fragments(new_text)
    new_text = remove_parentheticals(new_text)
    new_text = remove_prepositional_phrases(new_text)
    return new_text


def get_text_output(text: list):
    final_text = []
    for sentence in text:
        final_text.append(' '.join([word['word'] for word in sentence]))
    return ' '.join(final_text)


In [18]:
# getting pos tags for words

stanza.download('hi')
nlp = stanza.Pipeline('hi')

def get_pos_tags(text, pipeline):
    text_tagged = []
    
    doc = pipeline(text)
    for sentence in doc.sentences:
        sent = []
        for word in sentence.words:
            sent.append({"word": word.text, "label": word.xpos})
        text_tagged.append(sent) 
        
    return text_tagged   
    
# Example output
hindi_text = "अफ्रीका का वन्यजीवन दुनिया भर में अपनी विविधता और अनोखेपन के लिए प्रसिद्ध है। यहाँ के विस्तृत सवाना, घने जंगल और विशाल रेगिस्तान असंख्य प्रजातियों का घर हैं। अफ्रीका में शेर, चीता, तेंदुआ, हाथी, गैंडा और भैंस जैसे बड़े जानवरों से लेकर जिराफ, ज़ेब्रा और हिप्पोपोटामस तक कई अद्भुत प्राणी पाए जाते हैं। इसके अलावा, यहाँ पक्षियों की भी हजारों प्रजातियाँ देखने को मिलती हैं। अफ्रीका के राष्ट्रीय उद्यान और संरक्षित क्षेत्र, जैसे कि मसाई मारा, क्रूगर नेशनल पार्क और सेरेनगेटी, वन्यजीवन संरक्षण के महत्वपूर्ण केंद्र हैं। हालाँकि, शिकार और वनों की कटाई के कारण कई प्रजातियाँ खतरे में हैं। इनके संरक्षण के लिए अनेक प्रयास किए जा रहे हैं, ताकि भविष्य की पीढ़ियाँ भी इस अद्भुत वन्यजीवन का आनंद ले सकें।"
tagged_hindi_text = get_pos_tags(hindi_text, pipeline=nlp)
for sentence in tagged_hindi_text:
    print(sentence)

2025-01-27 17:01:58 INFO: Downloaded file to C:\Users\shubh\stanza_resources\resources.json
2025-01-27 17:01:58 INFO: Downloading default packages for language: hi (Hindi) ...
2025-01-27 17:01:59 INFO: File exists: C:\Users\shubh\stanza_resources\hi\default.zip
2025-01-27 17:02:02 INFO: Finished downloading models and saved to C:\Users\shubh\stanza_resources
2025-01-27 17:02:02 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-01-27 17:02:03 INFO: Downloaded file to C:\Users\shubh\stanza_resources\resources.json
2025-01-27 17:02:03 INFO: Loading these models for language: hi (Hindi):
| Processor | Package       |
-----------------------------
| tokenize  | hdtb          |
| pos       | hdtb_charlm   |
| lemma     | hdtb_nocharlm |
| depparse  | hdtb_charlm   |

2025-01-27 17:02:03 INFO: Using device: cuda
2025-01-27 17:02:03 INFO: Loading: tokenize
c:\Users\shubh\AppData\Local\Programs\Python\Python311\Lib\site-packages\stanza\models\tokenization\trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the fun

[{'word': 'अफ्रीका', 'label': 'NNP'}, {'word': 'का', 'label': 'PSP'}, {'word': 'वन्यजीवन', 'label': 'NN'}, {'word': 'दुनिया', 'label': 'NN'}, {'word': 'भर', 'label': 'RP'}, {'word': 'में', 'label': 'PSP'}, {'word': 'अपनी', 'label': 'PRP'}, {'word': 'विविधता', 'label': 'NN'}, {'word': 'और', 'label': 'CC'}, {'word': 'अनोखेपन', 'label': 'NN'}, {'word': 'के', 'label': 'PSP'}, {'word': 'लिए', 'label': 'PSP'}, {'word': 'प्रसिद्ध', 'label': 'JJ'}, {'word': 'है', 'label': 'VM'}, {'word': '।', 'label': 'SYM'}]
[{'word': 'यहाँ', 'label': 'PRP'}, {'word': 'के', 'label': 'PSP'}, {'word': 'विस्तृत', 'label': 'JJ'}, {'word': 'सवाना', 'label': 'NN'}, {'word': ',', 'label': 'SYM'}, {'word': 'घने', 'label': 'JJ'}, {'word': 'जंगल', 'label': 'NN'}, {'word': 'और', 'label': 'CC'}, {'word': 'विशाल', 'label': 'JJ'}, {'word': 'रेगिस्तान', 'label': 'NN'}, {'word': 'असंख्य', 'label': 'JJ'}, {'word': 'प्रजातियों', 'label': 'NN'}, {'word': 'का', 'label': 'PSP'}, {'word': 'घर', 'label': 'NN'}, {'word': 'हैं', 'lab

1st Approach: Using LLMs to find deletions and delete them

In [19]:
summary_prompt = "Generate a summary of the following piece of text in Hindi Language in %d words. Do not use any words fron any other language. Be concise and informative. Do not generate any text other than the summary. The text starts from below:\n%s"
compression_prompt = "I want to compress the below Hindi text by removing parts that are not required to get the complete meaning of the text. Please delete the words and phrases that can be deleted without affecting the meaning of the text. Do not generate words other than the compressed summary. The summary starts from below:\n%s"
add_more_words_prompt = "I have a piece of Hindi text below:\n%s\nCould you please add %d more words to the following summary so that it becomes more informative and complete? Do not use any words other than Hindi language. Generate only the summary, and no other words. The summary starts from below:\n%s"
system_role = "You are an expert Hindi linguist."


def generate_summary_pipeline(text, summary_prompt, compression_prompt, add_more_words_prompt, compression_factor):
    words_in_text = len(text.split())
    words_in_final_summary = int(words_in_text * compression_factor)
    global system_role
    
    tokenizer  = AutoTokenizer.from_pretrained("nvidia/Nemotron-4-Mini-Hindi-4B-Instruct")
    model = AutoModelForCausalLM.from_pretrained("nvidia/Nemotron-4-Mini-Hindi-4B-Instruct").cuda()
    
    # generate summary
    messages=[
                {"role": "system", "content": system_role},
                {"role": "user", "content": summary_prompt % (words_in_final_summary, text)},
            ]
    tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")
    tokenized_chat = tokenized_chat.cuda()
    model = model.cpu()
    tokenized_chat = tokenized_chat.cpu()
    outputs = outputs.cpu()
    first_summary = tokenizer.decode(outputs[0])
    
    tokenizer  = AutoTokenizer.from_pretrained("nvidia/Nemotron-4-Mini-Hindi-4B-Instruct")
    model = AutoModelForCausalLM.from_pretrained("nvidia/Nemotron-4-Mini-Hindi-4B-Instruct").cuda()
    
    # compress the summary
    messages=[
                {"role": "system", "content": system_role},
                {"role": "user", "content": compression_prompt % first_summary},
            ]
    tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")
    tokenized_chat = tokenized_chat.cuda()
    model = model.cpu()
    tokenized_chat = tokenized_chat.cpu()
    outputs = outputs.cpu()
    first_summary = tokenizer.decode(outputs[0])

    print("Loop 0:", first_summary)

    current_summary_words = len(first_summary.split())
    new_summary = first_summary
    
    loops = 0
    while abs(words_in_final_summary - current_summary_words) > 20:
        print("Required words:", words_in_final_summary)
        print("Current words:", current_summary_words)
        
        tokenizer  = AutoTokenizer.from_pretrained("nvidia/Nemotron-4-Mini-Hindi-4B-Instruct")
        model = AutoModelForCausalLM.from_pretrained("nvidia/Nemotron-4-Mini-Hindi-4B-Instruct").cuda()
        
        # add words to compressed summary
        messages=[
                {"role": "system", "content": system_role},
                {"role": "user", "content": add_more_words_prompt % (text, words_in_final_summary - current_summary_words, new_summary)},
            ]
        tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")
        tokenized_chat = tokenized_chat.cuda()
        model = model.cpu()
        tokenized_chat = tokenized_chat.cpu()
        outputs = outputs.cpu()
        new_summary = tokenizer.decode(outputs[0])
        
        
        tokenizer  = AutoTokenizer.from_pretrained("nvidia/Nemotron-4-Mini-Hindi-4B-Instruct")
        model = AutoModelForCausalLM.from_pretrained("nvidia/Nemotron-4-Mini-Hindi-4B-Instruct").cuda()
        
        # compress the summary
        messages=[
                {"role": "system", "content": system_role},
                {"role": "user", "content": compression_prompt % new_summary},
            ]
        tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")
        tokenized_chat = tokenized_chat.cuda()
        model = model.cpu()
        tokenized_chat = tokenized_chat.cpu()
        outputs = outputs.cpu()
        new_summary = tokenizer.decode(outputs[0])
        
        current_summary_words = len(new_summary.split())
        loops += 1
        
        print(f"Loop {loops}:", new_summary)
        
    return (loops, new_summary)


In [20]:
# load the data
rows_provided = 10

data = pandas.read_excel(r'C:\Users\shubh\Desktop\HDTB_pre_release_version-0.05\CleanHindi2kMT5_xlsum (3).xlsx', nrows=rows_provided)
data

,id,text,target,generated0.1,generated_openhathi,generatedgemma
0,0,"सभी मित्रों को जेएमसी। की बाइबल वाणी.कॉम में, ...",बाइबिल दिन की बाइबिल पद्य बाइबिल से 100 वादे,बाइबल के अनमोल वचनों को हम भी सुने और अपने जीव...,बाइबल में दिखाया गया है कि दुनिया के लिए सबसे ...,"सभी मित्रों को जेएमसी। की बाइबल वाणी.कॉम में,..."
1,1,ब्रेस्ट लॉट हर एरिया। तो दोस्तों मोनिका की ओर ...,यह एक बुद्धिमान व्यक्ति के बारे में ऑडियो कहान...,बीबीसी हिंदी सेवा के विशेष कार्यक्रम 'एक मुलाक...,मोनिका ने उन सभी को सुना था जिसे उसे सुनाना था...,ब्रेस्ट लॉट हर एरिया। तो दोस्तों मोनिका की ओर...
2,2,आप सभी को जेएमसी की बाइबल वाणी.कॉम में आप सभी ...,मेरा नवीनतम एपिसोड देखें! बाइबिल सिद्धांत,जब तक हमारे पास पवित्र शास्त्र बाइबल का सिद्धा...,इस पाठ के उदाहरणों में से एक का उपयोग करके मैं...,आज हम सीखेंगे कि पवित्र शास्त्र बाइबल का सिद्ध...
3,3,"हैलो दोस्तों, कैसे है? आप सभी विश्वास करती हूँ...",यह ऑडियो स्वेट मार्डन द्वारा लिखी गई किताब एवर...,हमारे दोस्तों ने मेरी ऑडियो स्वेट मॉडर्न के बा...,अगर आप अपने बारे में एक व्यक्ति को महसूस कराते...,## आन न में आपको मजब्रान मजब्रान आन में आप ...
4,4,हैलो दोस्तों। आज इससे कठिन समय में। जब सब तरफ ...,"दैनिक ध्यान, भगवान के साथ समय, यीशु के साथ समय...",परमिशन सर और रीती के बारे में क्या आप जानते है...,"100 के लिए, मुझे लगता है कि आपको तय करना होगा ...","हाँ, आप को इस संकट में परमेश्वर के वचन को सुना..."
5,5,हिंदी के एक सुप्रसिद्ध लेखक ने क्या खूब कहा है...,यह कहानी हमें आखिरी सांस तक कोशिश करने के लिए ...,"कोशिश कर हल निकलेगा आज नहीं तो कल निकलेगा, मेह...","एक कहानी सुनाने वाली अपनी कहानी के दौरान, मोनि...",कुछ लिखने के लिए एक सुप्रसिद्ध लेखक ने कहानी म...
6,6,"प्रेस स्लॉट ह ले लिया सभी को जेम्स की दोस्तों,...","दैनिक ध्यान, भगवान के साथ समय, यीशु के साथ समय...",जेम्स और उनके साथियों के बारे में हमने कई बाते...,"जो बात सभी को एक साथ ला सकती है, उससे आगे भी ज...","प्रेस स्लॉट ह ले लिया सभी को जेम्स की दोस्तों,..."
7,7,हैलो फ्रेंड्स कैसे है? आप सभी विश्वास करते हैं...,यह ऑडियो स्वेट मार्डन द्वारा लिखी गई किताब एवर...,स्वेट मॉडर्न की लिखी गई महान प्रति एव्रीमैन आ ...,"हैलो, दोस्तों!",यह संक्षमtext आपको बताता है कि मित्रों के साथ ...
8,8,तुम सत्य को जानोगे। सत्य तुम्हें स्वतंत्र करेग...,आपको सच्चाई का पता चलेगा और सच्चाई आपको मोनिका...,बीबीसी हिंदी सेवा के विशेष कार्यक्रम 'सत्य और ...,मानव जीवन का अर्थ सत्य की खोज करना है। सत्य की...,इस संक्षमSummary में आप सत्य को जानने के बारे ...
9,9,"अवसर को बहुमूल्य जानों, क्योंकि दिन बुरे है। न...",अवसर का लाभ उठाएं क्योंकि दिन खराब हैं,बीबीसी हिंदी सेवा के खबरी चैनल के संपादक राजेश...,अवसर को बहु-मूल्यवान जानों के रूप में समझें क्...,"अवसर को बहुमूल्य जानों क्योंकि दिन बुरे है, इस..."


In [21]:
summaries = []
for text in data['text']:
    print("Text:", text)
    print()
    summary = generate_summary_pipeline(text, summary_prompt, compression_prompt, add_more_words_prompt, 0.2)
    summaries.append(summary)
    print("Summary:", summary)
    print()
    print()

data['llm_deletions'] = summaries
data.to_excel(r'C:\Users\shubh\Desktop\HDTB_pre_release_version-0.05\CleanHindi2kMT5_xlsum (3).xlsx', index=False)

Text: सभी मित्रों को जेएमसी। की बाइबल वाणी.कॉम में, मैं मोनिका आप सभी का स्वागत करती हूँ। बाइबल विश्व की सबसे अद्भुत पुस्तक है जिसे विश्व की सबसे ज्यादा भाषाओं में अनुवाद किया गया और प्रकाशित किया गया है। बाइबल की शिक्षाओं को सार्वभौमिक रूप से सत्य माना गया है क्योंकि बाइबल के अनमोल वचनों ने दुनिया के प्रसिद्ध लोगों को प्रभावित किया है, जिसमें से हमारे भारत के महात्मा गाँधी जी भी हुए। यहाँ पर हम बाइबल के कुछ अनमोल वचनों को सुनने जा रहे हैं। परमेश्वर के एक श्रेष्ठ दास ने इस प्रकार से कहा, हम वह बाइबल है जिसे दुनिया पढ़ती है। हम वो पंथ हैं जिनकी दुनिया को आवश्यकता है। हम वो उपदेश हैं जिन पर दुनिया ध्यान दे रही है। आइये बाइबल के आज इन अनमोल वचनों को हम भी सुने और अपने जीवन में आशीष ग्रहण करें। यदि परमेश्वर हमारी ओर हैं तो हमारा विरोधी कौन हो सकता है? जो यत्न से भलाई करता है वह औरों की प्रसन्नता खोजता है, परंतु जो दूसरों की बुराइ का खोजी होता है, उसी पर बुरा ई आप पड़ती है। पवित्र शास्त्र बाइबल में बताया गया है जो व्यक्ति दूसरों की भलाई चाहता है और दूसरों के जीवन में प्रसन्नता लाना चाहता है, 

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, you requested 16868 tokens (13604 in the messages, 3264 in the completion). Please reduce the length of the messages or completion.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [28]:
def generate_summary_pipeline_logic(text, summary_prompt, add_more_words_prompt, compression_factor):
    words_in_text = len(text.split())
    words_in_final_summary = words_in_text * compression_factor
    
    tokenizer  = AutoTokenizer.from_pretrained("nvidia/Nemotron-4-Mini-Hindi-4B-Instruct")
    model = AutoModelForCausalLM.from_pretrained("nvidia/Nemotron-4-Mini-Hindi-4B-Instruct").cuda()
    
    # generate summary
    messages=[
                {"role": "system", "content": system_role},
                {"role": "user", "content": summary_prompt % (words_in_final_summary, text)},
            ]
    tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")
    tokenized_chat = tokenized_chat.cuda()
    model = model.cpu()
    tokenized_chat = tokenized_chat.cpu()
    outputs = outputs.cpu()
    first_summary = tokenizer.decode(outputs[0])

    # compress the summary
    processed_text = get_pos_tags(first_summary, pipeline=nlp)
    deleted_required_parts = delete_required_parts(processed_text)
    first_summary = get_text_output(deleted_required_parts)

    current_summary_words = len(first_summary.split())
    new_summary = first_summary
    
    loops = 0
    print("Loop:", loops)
    while abs(words_in_final_summary - current_summary_words) > 10:
        print("Required words:", words_in_final_summary)
        print("Current words:", current_summary_words)
        
        tokenizer  = AutoTokenizer.from_pretrained("nvidia/Nemotron-4-Mini-Hindi-4B-Instruct")
        model = AutoModelForCausalLM.from_pretrained("nvidia/Nemotron-4-Mini-Hindi-4B-Instruct").cuda()
        
        # add words to compressed summary
        messages=[
                {"role": "system", "content": system_role},
                {"role": "user", "content": add_more_words_prompt % (text, words_in_final_summary - current_summary_words, new_summary)},
            ]
        tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")
        tokenized_chat = tokenized_chat.cuda()
        model = model.cpu()
        tokenized_chat = tokenized_chat.cpu()
        outputs = outputs.cpu()
        new_summary = tokenizer.decode(outputs[0])

        # compress the summary
        processed_text = get_pos_tags(new_summary, pipeline=nlp)
        deleted_required_parts = delete_required_parts(processed_text)
        new_summary = get_text_output(deleted_required_parts)
        
        current_summary_words = len(new_summary.split())
        loops += 1
        print("Loop:", loops)
        
    return (loops, new_summary)

In [ ]:
summaries = []
for text in data['text']:
    print("Text:", text)
    print()
    summary = generate_summary_pipeline_logic(text, summary_prompt, add_more_words_prompt, 0.2)
    summaries.append(summary)
    print("Summary:", summary)
    print()
    print()

data['logic_deletions'] = summaries
data.to_excel(r'C:\Users\shubh\Desktop\HDTB_pre_release_version-0.05\CleanHindi2kMT5_xlsum (3).xlsx', index=False)